# Load raw dataset
A medium-sized dataset is provided (rather than 12 GB full dataset as is in AWS) for data wrangling in the spark cluster of IBM Watson Studio. 


In [1]:
# Load raw data to spark

import ibmos2spark, os
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.private.us.cloud-object-storage.appdomain.cloud',
    'service_id': 'iam-ServiceId-b314834d-5443-4359-8efa-88fb1ca94789',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'NfS0KqvIB-QZO6PoO6p1UYPdzthvM2t3mOeNGUatbbnC'
}

configuration_name = 'os_c4ce378e6d4243b4bee4d7f946ea0bff_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df_data_1 = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-crgpbxoft3rjye'))
df_data_1.take(5)

NameError: name 'sc' is not defined

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df_data_1 = spark.read.json('medium-sparkify-event-data.json')
df_data_1.take(5)

22/08/09 09:25:11 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.23 instead (on interface en0)
22/08/09 09:25:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/09 09:25:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98'),
 Row(artist='Afroman', auth='Logged In', firstName='Maverick', gender='M', 

# Data cleanning and exploration 
In this section, data cleaning is completed including checking size of dataset, columns, invalid or missing data, drop duplicates, exploratory visualization, categorical data processing, and so forth. 

In [3]:
from pyspark.sql import Row
from pyspark.sql.functions import col,isnan, when, count
import matplotlib.pyplot as plt

In [4]:
df_data_1.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
df_data_1.show()

+--------------------+----------+---------+------+-------------+---------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|      auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+----------+---------+------+-------------+---------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Martin Orford| Logged In|   Joseph|     M|           20|  Morales|597.55057| free|  Corpus Christi, TX|   PUT|       NextSong|1532063507000|      292|       Grand Designs|   200|1538352011000|"Mozilla/5.0 (Mac...|   293|
|   John Brown's Body| Logged In|   Sawyer|     M|           74|   Larson|380.21179|

In [6]:
print((df_data_1.count(), len(df_data_1.columns)))

(543705, 18)


In [7]:
df_data_1.select('page').distinct().show(100, False)

+-------------------------+
|page                     |
+-------------------------+
|Cancel                   |
|Submit Downgrade         |
|Thumbs Down              |
|Home                     |
|Downgrade                |
|Roll Advert              |
|Logout                   |
|Save Settings            |
|Cancellation Confirmation|
|About                    |
|Settings                 |
|Login                    |
|Add to Playlist          |
|Add Friend               |
|NextSong                 |
|Thumbs Up                |
|Help                     |
|Upgrade                  |
|Error                    |
|Submit Upgrade           |
|Submit Registration      |
|Register                 |
+-------------------------+



In [8]:
df_data_1.select('song').distinct().count()

80293

In [9]:
df_data_1.agg({'length': 'max'}).collect()[0]

Row(max(length)=3024.66567)

# Create Churn column based on "page" column

In [10]:
cancel_set = ['Cancel', 'Submit Downgrade', 'Thumbs Down', 'Downgrade', 'Cancellation Confirmation']
df_data_1 = df_data_1.withColumn('Churn', when(df_data_1.page.isin(cancel_set), 1).otherwise(0))
df_data_1.show()

+--------------------+----------+---------+------+-------------+---------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+
|              artist|      auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|Churn|
+--------------------+----------+---------+------+-------------+---------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+
|       Martin Orford| Logged In|   Joseph|     M|           20|  Morales|597.55057| free|  Corpus Christi, TX|   PUT|       NextSong|1532063507000|      292|       Grand Designs|   200|1538352011000|"Mozilla/5.0 (Mac...|   293|    0|
|   John Brown's Body| Logged In|   Sawyer|     M|          

# Remove invalid and missing values
1. Intuitively, we do not think the leave or stay of a customer would be affected by his/her name. So the columns "firstName" and "lastName" can be simply dropped.
2. Based on the aggregation results of groupBy('Churn', 'artist'), groupBy('Churn', 'song'), and column "length", all cases of Churn==1 have null artist, song, and length, so churn rate is also not related to them in this dataset. Here we drop columns "artist", "song", and "length" as well.
3. After dropping unused columns, we filter out rows with null value. 

In [11]:
# Count missing values for each column
df_data_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_data_1.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+-----+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|  song|status| ts|userAgent|userId|Churn|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+-----+
|110828|   0|    15700| 15700|            0|   15700|110828|    0|   15700|     0|   0|       15700|        0|110829|     0|  0|    15700|     0|    0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+-----+



In [12]:
count_churn_0 = df_data_1.groupBy('Churn', 'artist').count().filter(df_data_1.Churn == 0)
count_churn_1 = df_data_1.groupBy('Churn', 'artist').count().filter(df_data_1.Churn == 1)
count_churn_0.show()
count_churn_1.show()

+-----+--------------------+-----+
|Churn|              artist|count|
+-----+--------------------+-----+
|    0|           The Kooks|  467|
|    0|             Bacilos|  121|
|    0|Brigitte Bardot /...|    7|
|    0|         Passion Pit|  345|
|    0|        Supersuckers|    8|
|    0|         Los Bunkers|   46|
|    0|               Opeth|  117|
|    0|         State Radio|   40|
|    0|                 Mud|   48|
|    0|Jack Johnson / Ka...|   21|
|    0|   Story Of The Year|  100|
|    0|    Peter And Gordon|   29|
|    0|    David Arkenstone|   24|
|    0|          Eskimo Joe|   30|
|    0|         Brian Regan|   17|
|    0|     Massive TÃÂ¶ne|    4|
|    0|       Elvis Presley|   33|
|    0|         Boney James|   57|
|    0|      The Fire Theft|   17|
|    0|Major Lazer / Vyb...|   16|
+-----+--------------------+-----+
only showing top 20 rows

+-----+------+-----+
|Churn|artist|count|
+-----+------+-----+
|    1|  null| 9037|
+-----+------+-----+



In [13]:
count_churn_0 = df_data_1.groupBy('Churn', 'song').count().filter(df_data_1.Churn == 0)
count_churn_1 = df_data_1.groupBy('Churn', 'song').count().filter(df_data_1.Churn == 1)
count_churn_0.show()
count_churn_1.show()

+-----+--------------------+-----+
|Churn|                song|count|
+-----+--------------------+-----+
|    0|               Reign|   10|
|    0|    Ne Me Quitte Pas|    2|
|    0|    Snow White Queen|    9|
|    0|            Auto-Dub|   15|
|    0|          Release Me|   20|
|    0|The Desperate Kin...|    3|
|    0|             Dragula|   29|
|    0|      Zamba de Usted|    1|
|    0|May The Words Of ...|    6|
|    0|I MÃÂ¶rkret Med Dej|   11|
|    0|Searching (So Muc...|    4|
|    0|Hypnotized (featu...|   24|
|    0|           Entertain|    6|
|    0|Diverse City (Wel...|    3|
|    0|My Drink (I Remem...|   14|
|    0|Machine Gun Funk ...|   19|
|    0|               Saxon|    5|
|    0|            Over You|   15|
|    0|Rubber & Soul (An...|    2|
|    0|          Fast-Track|   16|
+-----+--------------------+-----+
only showing top 20 rows

+-----+----+-----+
|Churn|song|count|
+-----+----+-----+
|    1|null| 9037|
+-----+----+-----+



In [14]:
count_churn_0 = df_data_1.filter(df_data_1.Churn == 0).select('length').show()
count_churn_1 = df_data_1.filter(df_data_1.Churn == 1).select('length').show()

+---------+
|   length|
+---------+
|597.55057|
|380.21179|
|202.37016|
|     null|
|194.53342|
|138.29179|
|     null|
|     null|
|     null|
|     null|
|     null|
|220.39465|
|     null|
|201.50812|
|357.66812|
|256.96608|
|164.23138|
|251.48036|
|     null|
|157.67465|
+---------+
only showing top 20 rows

+------+
|length|
+------+
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
+------+
only showing top 20 rows



In [15]:
cols = ('artist' ,'firstName', 'lastName', 'length', 'song')
df_drop = df_data_1.drop(*cols)
df_drop.printSchema()

root
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- Churn: integer (nullable = false)



In [16]:
# Remove null records
df_drop = df_drop.na.drop()
df_drop.count()

528005